In [63]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)
financial_data.replace('в ноль', '0', regex=True, inplace=True)

In [64]:
managers = prolongations["AM"].unique()

In [65]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [66]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [67]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

In [68]:
for manager in managers:
    for month in months_to_calc:
        # получаем все проекты, которые были завершены в предыдущих месяцах
        prev_prev_month_index = all_months.index(month) - 2
        prev_month_index = all_months.index(month) - 1
        prev_prev_month = all_months[prev_prev_month_index]
        prev_month = all_months[prev_month_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        prev_prev_month_completed_projects = manager_completed_projects[manager_completed_projects['month'] == prev_prev_month]
        prev_month_completed_projects = manager_completed_projects[manager_completed_projects['month'] == prev_month]
        
        k2_up_sum = 0
        k2_down_sum = 0
        
        print(month)
        # print("просто проекты сделанные менеджером:")
        print(prev_prev_month_completed_projects)
        ids = prev_prev_month_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
        # print("финансовые данные по проектам:")
        # print(projects_financial_data)
        
        for id, project in projects_financial_data.iterrows():
            for check_month in range(0, prev_prev_month_index + 1):
                if project[capitalize_first_letter(all_months[check_month])] == "стоп" or project[capitalize_first_letter(all_months[check_month])] == "end":
                    # тогда исключаем такой проект из расчета пролонгаций:
                    projects_financial_data.drop(id, inplace=True)
        
        required_months = all_months[0: all_months.index(month) + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months.append('id')

        # print("проекты без стопов:")
        # print(projects_financial_data)
        projects_financial_data.replace('стоп', '0', regex=True, inplace=True)
        projects_financial_data.replace('end', '0', regex=True, inplace=True)
        
        projects_financial_data = projects_financial_data[required_months]
        projects_financial_data = projects_financial_data.astype(float)
        # print()
        
        projects_financial_data_grouped = projects_financial_data.groupby("id").sum()
        
        

        # print("проекты без стопов:")
        # print(projects_financial_data)
        
            # if float(project[capitalize_first_letter(prev_prev_month)]):
            #     k2_up_sum += float(project[capitalize_first_letter(prev_prev_month)])
            #     k2_down_sum += float(project[capitalize_first_letter(prev_month)])
            #     print(f"Проект {project['id']} в месяце {prev_prev_month} имеет значение {project[capitalize_first_letter(prev_prev_month)]}")
        # for id in prev_prev_month_completed_projects['id']:
        #     projects = financial_data.loc[financial_data['id'] == id]
        #     for check_month in range(0, prev_prev_month_index + 1):
        #         disbursement = projects[capitalize_first_letter(all_months[check_month])]
        #         print(len(disbursement))
        #         print(disbursement)
            # for _, project in projects.iterrows():
            #     if float(project[capitalize_first_letter(prev_month)]):
            #         print(f"Проект {id} в месяце {prev_month} имеет значение {project[capitalize_first_letter(prev_month)]}")
                
# for months in months_to_calc:
#     prev_prev_month = all_months[all_months.index(months) - 2]
#     prev_month = all_months[all_months.index(months) - 1]
# 
#     prev_prev_month_completed_projects = prolongations[prolongations['month'] == prev_prev_month]
#     prev_month_completed_projects = prolongations[prolongations['month'] == prev_month]
    

январь 2023
     id        month                            AM
0    42  ноябрь 2022  Васильев Артем Александрович
1   453  ноябрь 2022  Васильев Артем Александрович
5   600  ноябрь 2022  Васильев Артем Александрович
6   657  ноябрь 2022  Васильев Артем Александрович
7   594  ноябрь 2022  Васильев Артем Александрович
8   665  ноябрь 2022  Васильев Артем Александрович
13  579  ноябрь 2022  Васильев Артем Александрович
15  633  ноябрь 2022  Васильев Артем Александрович
17  369  ноябрь 2022  Васильев Артем Александрович
19  592  ноябрь 2022  Васильев Артем Александрович
20  666  ноябрь 2022  Васильев Артем Александрович
21  620  ноябрь 2022  Васильев Артем Александрович
проекты без стопов:
      id        Причина дубля Ноябрь 2022 Декабрь 2022 Январь 2023  \
0     42                  NaN    36220.00          NaN         NaN   
4    665                  NaN    10000.00          NaN         NaN   
8    579                  NaN           0          NaN         NaN   
9    592  первая часть оп